In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import Machine, generate_objects, BunchTracker, track, compare_traces, BeamTracker, plot3Dtraces
from test_tools import compare_beam_projections, plot_debug_data, plot_FIR_coefficients

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors.multiplication import ChargeWeighter
from PyHEADTAIL.feedback.processors.linear_transform import Averager
from PyHEADTAIL.feedback.processors.misc import Bypass
from PyHEADTAIL.feedback.processors.register import Register, TurnDelay, TurnFIRFilter
from PyHEADTAIL.feedback.processors.convolution import Lowpass, Gaussian, FIRFilter
from PyHEADTAIL.feedback.processors.resampling import DAC, HarmonicADC, BackToOriginalBins, Upsampler
from PyHEADTAIL.feedback.processors.addition import NoiseGenerator
np.random.seed(0)

# 009 Injection error damping

This tests injection error damping by using a model for the original LHC damper system (ADT). The model still in progress and might not be correct!

## Basic parameters and elements for the simulations

In [ ]:
%%capture

n_macroparticles = 1000
n_slices = 20
n_segments = 5
n_sigma_z = 6
n_sigma_z = 6

n_turns = 100
# n_turns = 12

machine = Machine(n_segments= n_segments)

# harmonic frequency of the bunches (f_RF/every_n_bucket_filled)
f_RF = 1./(machine.circumference/c/(float(machine.h_RF)))
f_harmonic = f_RF/10.


first_index = 10 #[buckets]
batch_spacing = 8  #[buckets]
# batch_spacing = 10  #[buckets]
n_batches = 3
# n_batches = 3
n_bunches_per_batch = 72
# n_bunches_per_batch = 6
bunch_spacing = 1 #[buckets]
LHC_gap = 38

batch_separation = batch_spacing+n_bunches_per_batch* bunch_spacing

filling_scheme = []

for i in xrange(n_bunches_per_batch):
    filling_scheme.append(first_index + i * bunch_spacing)

first_index = np.max(filling_scheme) + LHC_gap
for j in xrange(n_batches):
    for i in xrange(n_bunches_per_batch):
        filling_scheme.append(first_index + i * bunch_spacing + j*batch_separation)


print filling_scheme

beam_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles, n_slices,n_sigma_z,
                                                             filling_scheme=filling_scheme, matched=False)

In [ ]:
print 'f_RF: ' + str(f_RF)
print 'f_harmonic: ' + str(f_harmonic)
print('Number of bunches: ' + str(len(beam_ref.split())))

## Initial bunch kick
The bunhes are initial kicks are estimated from numerical MKI kicker waveform data. Data from Gerd Kotzian's archive.

In [ ]:
# numerical data for the injection error
kick_data = np.loadtxt('./injection_error_input_data/field_May24_2007_Mag5_7kV5_2.CSV',delimiter=',')

fig, ax = plt.subplots(1)
kick_data[:,1] = kick_data[:,1]-0.95
kick_data[:,1] = kick_data[:,1]/np.max(kick_data[:,1])
neg_map = (kick_data[:,1] < 0.)
kick_data[neg_map,1] = 0.

ax.plot(kick_data[:,0]*1e6,kick_data[:,1])
ax.set_ylim(0,1.0)
ax.set_xlim(0,10.0)

plt.show()

# kick_data = np.stack((kick_data[0::2], kick_data[1::2]), axis=-1)
# kick_data[:,0] = (kick_data[:,0]-0.95)*1e-6
# kick_data[:,1] = kick_data[:,1]/10.

In [ ]:
bunch_list = beam_ref.split()

n_bunches = len(bunch_list)

amplitude = 10e-3

t0 = bunch_list[n_bunches - (n_bunches_per_batch)].bucket_id[0]*machine.circumference/machine.h_bunch/c
for i in xrange(n_bunches):
    if i < (n_bunches - (n_bunches_per_batch)):
        z_location = bunch_list[i].bucket_id[0]*machine.circumference/machine.h_bunch
        
        bunch_list[i].x = bunch_list[i].x + amplitude*np.interp((z_location/c-t0), kick_data[:,0],kick_data[:,1])
        bunch_list[i].y = bunch_list[i].y + amplitude*np.interp((z_location/c-t0), kick_data[:,0],kick_data[:,1])
#         bunch_list[i].x = bunch_list[i].x + amplitude
#         bunch_list[i].y = bunch_list[i].y + amplitude
    else:
        z_location = bunch_list[i].bucket_id[0]*machine.circumference/machine.h_bunch


beam_ref = sum(bunch_list)

## Special signal processor: SignalHolder

Special signal processor, which holds the signal between the gaps of the injected batches

In [ ]:
from PyHEADTAIL.feedback.core import default_macros

class SignalHolder(object):
    def __init__(self, hold_pattern, **kwargs):
        self._hold_pattern = hold_pattern
        
        self.extensions = []
        self._macros = [] + default_macros(self, 'SignalHolder', **kwargs)
        
        self._value_maps = []
        self._hold_maps = []
        
        value_indexes = self._hold_pattern[self._hold_pattern > 0]
        
        for i, idx in enumerate(value_indexes):
            self._value_maps.append(np.where(self._hold_pattern==idx)[0][0])
            self._hold_maps.append(np.where(self._hold_pattern==-idx)[0])
        
        
    def process(self, parameters, signal, *args, **kwargs):
        signal_out = np.copy(signal)
        
        for source_idx, target_map in zip(self._value_maps, self._hold_maps):
            signal_out[target_map] = signal[source_idx]
            
        return parameters, signal_out
        

Determines holding scheme for the SignalHolder

In [ ]:
extra_adc_bins = 10

max_bucket = np.max(filling_scheme)
min_bucket = np.min(filling_scheme)

n_bins = (max_bucket - min_bucket) + 1 
n_bins = n_bins + 2*extra_adc_bins
holding_scheme = np.zeros(int(n_bins))

gap_1_value_idx = int(extra_adc_bins + 2*(n_bunches_per_batch)+ batch_spacing) - 1
gap_1_from = gap_1_value_idx + 1
gap_1_to = gap_1_from + int(batch_spacing)

gap_2_value_idx = int(extra_adc_bins + 3*(n_bunches_per_batch)+ 2*batch_spacing) - 1
gap_2_from = gap_2_value_idx + 1
gap_2_to = gap_2_from + int(batch_spacing)


holding_scheme[gap_1_value_idx] = 1
holding_scheme[gap_1_from:gap_1_to] = -1

holding_scheme[gap_2_value_idx] = 2
holding_scheme[gap_2_from:gap_2_to] = -2

## Feedback settings

In [ ]:
feedback_gain = 0.1
# feedback_gain = (0.1,0.4)

# delay (a number of turns) before the pickup signal is used to the correction kick calculations.
delay = 1

# a number of values used to calculate the correct signal
n_values = 2

RMS_noise_level = 1e-6


# feedback settings
fc=1e6 # The cut off frequency of the power amplifier
ADC_f = 40e9 # multiplier of the sampling rate from the harmonic frequency
ADC_n_samples = 50
ADC_bits = 16
ADC_range = (-3e-3,3e-3)

DAC_bits = 14
DAC_range = (-3e-3,3e-3)

RMS_noise_level = 1e-6

## Reference data
Tracks a bunch by using an ideal bunch-by-bunch feedback presented in the first test (001_ideal_feedbacks.ipynb). The data is used as baseline data for the detailed feedback model. 

In [ ]:
beam_ref_data = copy.deepcopy(beam_ref)
tracker_ref_data = BeamTracker(beam_ref_data)
slicer_ref_data = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_ref_data,processors_bunch_x,processors_bunch_y, mpi=True)
one_turn_map = [i for i in trans_map] + [feedback_map] #  + [long_map]

track(n_turns, beam_ref_data,one_turn_map ,tracker_ref_data)

## Detailed feedback model
This section contains an axample how to build a model for a LHC ADT type transverse feedback system.

**Disclimer:** it is not guaranteed that the following model describes any existing system.

### FIR filter coefficients for the bandwidth phase correction
The power amplifier phase and gain is corrected by using two FIR filters, which coefficients are loaded from an external file. 

Read more about the coefficients from:
http://be-op-lhc.web.cern.ch/sites/be-op-lhc.web.cern.ch/files/docs/ADT_operation.pdf
https://accelconf.web.cern.ch/accelconf/e08/papers/thpc122.pdf

In [ ]:
FIR_phase_filter = np.loadtxt('./injection_error_input_data/FIR_Phase_40MSPS.csv')
FIR_phase_filter = FIR_phase_filter/sum(FIR_phase_filter)
plot_FIR_coefficients(FIR_phase_filter)

FIR_gain_filter = np.loadtxt('./injection_error_input_data/FIR_Gain_120MSPS.csv')
FIR_gain_filter = FIR_gain_filter/sum(FIR_gain_filter)
plot_FIR_coefficients(FIR_gain_filter)

### Turn-by-turn FIR filter for the betatron phase correction
The betatron phase correction is implemented manually by using a turn-by-turn FIR filter. 


Read more about the coefficients from Ref. http://accelconf.web.cern.ch/AccelConf/IPAC2011/papers/mopo013.pdf.

In [ ]:
# the total (group) delay to the middle coefficient is four turns, i.e.

turn_notch_filter = [-1.,1.]

phase_shift_x = -5. * machine.Q_x * 2.* pi
turn_phase_filter_x = [-2. * np.sin(phase_shift_x)/(pi * 3.),
                   0,
                   -2. * np.sin(phase_shift_x)/(pi * 1.),
                   np.cos(phase_shift_x),
                   2. * np.sin(phase_shift_x)/(pi * 1.),
                   0,
                   2. * np.sin(phase_shift_x)/(pi * 3.)
                   ]

phase_shift_y = -5. * machine.Q_y * 2.* pi
turn_phase_filter_y = [-2. * np.sin(phase_shift_y)/(pi * 3.),
                   0,
                   -2. * np.sin(phase_shift_y)/(pi * 1.),
                   np.cos(phase_shift_y),
                   2. * np.sin(phase_shift_y)/(pi * 1.),
                   0,
                   2. * np.sin(phase_shift_y)/(pi * 3.)
                   ]


turn_FIR_filter_x = np.convolve(turn_notch_filter, turn_phase_filter_x)
turn_FIR_filter_y = np.convolve(turn_notch_filter, turn_phase_filter_y)

# turn_FIR_filter_x = turn_phase_filter_x
# turn_FIR_filter_y = turn_phase_filter_y

### The model
This is the actual damper model to be used in the simulations, which includes elements for digital signal processing (ADC, FIR filters and DAC) and power amplifier/kicker bandwidth limitation. It is assumed that the signal transmission from beam to ADC is perfect. A detailed model for a pickup signal processing could be build, but it is not done here in order to clarify the example.

In [ ]:
beam_detailed = copy.deepcopy(beam_ref)
tracker_detailed = BeamTracker(beam_detailed)
slicer_detailed = copy.deepcopy(slicer_ref)

processors_detailed_x = [
        Bypass(debug=False),
        ChargeWeighter(normalization = 'segment_average', debug=False),
#         NoiseGenerator(RMS_noise_level, debug=False),
        HarmonicADC(1*f_RF/10., ADC_bits, ADC_range,
                    n_extras=extra_adc_bins, debug=False),
        TurnFIRFilter(turn_FIR_filter_x, machine.Q_x, delay=1, debug=False),
        FIRFilter(FIR_phase_filter, zero_tap = 40, debug=True),
#         SignalHolder(holding_scheme, debug=True),
        Upsampler(3, [0,3,0], debug=False),
        FIRFilter(FIR_gain_filter, zero_tap = 32, debug=True),
        DAC(ADC_bits, ADC_range, debug=False),
        Lowpass(fc, f_cutoff_2nd=10*fc, debug=True),
        BackToOriginalBins(debug=False),
]

processors_detailed_y = [
        Bypass(debug=True),
        ChargeWeighter(normalization = 'segment_average', debug=False),
#         NoiseGenerator(RMS_noise_level, debug=False),
        HarmonicADC(1*f_RF/10., ADC_bits, ADC_range,
                    n_extras=extra_adc_bins, debug=False),
        TurnFIRFilter(turn_FIR_filter_y, machine.Q_y, delay = 1, debug=True),
        FIRFilter(FIR_phase_filter, zero_tap = 41, debug=True),
#         SignalHolder(holding_scheme, debug=True),
        Upsampler(3, [1,1,1], debug=False),
        FIRFilter(FIR_gain_filter, zero_tap = 32, debug=True),
        DAC(ADC_bits, ADC_range, debug=False),
        Lowpass(fc, f_cutoff_2nd=10*fc, debug=False),
        BackToOriginalBins(debug=False),
]


feedback_map = OneboxFeedback(feedback_gain,slicer_detailed,
                              processors_detailed_x,processors_detailed_y, mpi=True)
one_turn_map = [feedback_map] + [i for i in trans_map] # + [long_map]

track(n_turns, beam_detailed, one_turn_map, tracker_detailed)
plot_debug_data(processors_detailed_x, source = 'output')

In [ ]:
# compare_traces([tracker_ref_data,tracker_detailed],
#                ['Ideal bunch-by-slice\nfeedback', 'Damper model'], bunch_idx=-10)
# compare_beam_projections([ beam_ref_data, beam_detailed], 
#                ['Ideal bunch-by-slice\nfeedback', 'Damper model'])

Jani Komppula, CERN, 2017

In [ ]:


plot3Dtraces(tracker_ref_data,
               ['Ideal bunch-by-slice\nfeedback'], bunches = 130,first_turn=0)

plot3Dtraces(tracker_detailed,
               ['Minimal model'], bunches = [100,120,203])